<a href="https://colab.research.google.com/github/DaeSeokSong/image-processing/blob/feature%2FUnet-scar/Unet_Scar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reference

* *Paper*
>   [U-net](https://paperswithcode.com/paper/u-net-convolutional-networks-for-biomedical)
>   * [U-net++](https://paperswithcode.com/paper/unet-a-nested-u-net-architecture-for-medical)
>   * [ResUNet++](https://paperswithcode.com/paper/resunet-an-advanced-architecture-for-medical)

# Development enviroment

## Install

## Import

### Library

In [10]:
# U-net
import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from torchvision import transforms, datasets

# Image processing
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from google.colab.patches import cv2_imshow
from google.colab import output

# ETC
import os
import time

from PIL import Image

### Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%cd /content/gdrive/MyDrive/Models/GAN_Scar
!ls -al

# Image segmentation Modeling

## Grobal variable

In [9]:
# Path
MODEL_PATH = "/content/gdrive/MyDrive/Models/GAN_Scar"

TRAIN_SET_PATH = "/Dataset/train"
TEST_SET_PATH = "/Dataset/test"

IMAGES_PATH = '/images'
LABELS_PATH = '/labels'

# Train hyperparameter
LR = 1e-3
BATCH_SIZE = 4
EPOCH = 100

"""
GPU 사용이 가능하면 cuda 사용
아니면 CPU를 이용하여 학습
"""
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Class

In [ ]:
# torch.nn의 Module 클래스를 상속한, 커스텀 UNet 클래스
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()

## Funtion

In [ ]:
def imshow_waitkey_enter(image):
    cv2_imshow(image)

    time.sleep(0.5)
    input("Please press the Enter key to proceed\n")
    output.clear()

    pass

## Run

### Prepare dataset

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + TRAIN_SET_PATH + LABELS_PATH

# Load train images
os.chdir(image_path)
train_files = os.listdir(image_path)
train_files.sort()

train_images = []
for f in train_files: 
    train_images.append(cv2.imread(f))

# Load label images
os.chdir(label_path)
label_files = os.listdir(label_path)
label_files.sort()

label_images = []
for f in label_files: 
    label_images.append(cv2.imread(f))

Dataset = [(x, label_images[idx]) for idx, x in enumerate(train_images)]

### Model